# Semantic segmentation with LRASPP MobileNet v3 and OpenVINO
The [`torchvision.models`](https://pytorch.org/vision/stable/models.html) subpackage contains definitions of models for addressing different tasks, including: image classification, pixelwise semantic segmentation, object detection, instance segmentation, person keypoint detection, video classification, and optical flow. Throughout this notebook we will show how to use one of them.
The LRASPP model is based on the [Searching for MobileNetV3](https://arxiv.org/abs/1905.02244) paper. According to the paper, Searching for MobileNetV3, LR-ASPP or Lite Reduced Atrous Spatial Pyramid Pooling has a lightweight and efficient segmentation decoder architecture. he model is pre-trained on the [MS COCO](https://cocodataset.org/#home) dataset. Instead of training on all 80 classes, the segmentation model has been trained on 20 classes from the [PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/) dataset:
***background*, *aeroplane*, *bicycle*, *bird*, *boat*, *bottle*, *bus*, *car*, *cat*, *chair*, *cow*, *dining table*, *dog*, *horse*, *motorbike*, *person*, *potted plant*, *sheep*, *sofa*, *train*, *tv monitor***

More information about the model is available in the [torchvision documentation](https://pytorch.org/vision/main/models/lraspp.html)

### Table of content:
- [Prerequisites](#Prerequisites)
- [Download and prepare model](#Download-and-prepare-model)
- [Convert the original model to OpenVINO IR Format](#Convert-the-original-model-to-OpenVINO-IR-Format)
- [Show Results](#Show-results)
    - [Load an input image](#Load-an-input-image)
    - [Define a preprocessing and prepare an input data](#Define-a-preprocessing-and-prepare-an-input-data)
    - [Run an inference on the OpenVINO model](#Run-an-inference-on-the-OpenVINO-model)
    - [Run an inference on the PyTorch model to compare results](#Run-an-inference-on-the-PyTorch-model-to-compare-results)

## Prerequisites

In [ ]:
%pip install -q "torch==2.0.1" "torchvision==0.15.2"
%pip install -q datasets matplotlib
%pip install -q "openvino>=2023.1.0"

In [ ]:
from pathlib import Path

import openvino as ov
import torch

## Download and prepare model
Define width and height of the image that will be used by the network during inference. According to the input transforms function, the model is pre-trained on images with a height of 480 and width of 640.

In [ ]:
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

The torchvision module [provides](https://pytorch.org/vision/stable/models/generated/torchvision.models.segmentation.lraspp_mobilenet_v3_large.html#torchvision.models.segmentation.lraspp_mobilenet_v3_large) a ready to use set of functions for model class initialization. We will use `torchvision.models.segmentation.lraspp_mobilenet_v3_large`. You can directly pass pre-trained model weights to the model initialization function using weights enum LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1. 

In [ ]:
import torchvision.models as models


weights=models.segmentation.LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1
model = models.segmentation.lraspp_mobilenet_v3_large(weights=weights)

## Convert the original model to OpenVINO IR Format

To convert the original model to OpenVINO IR with `FP16` precision, use model conversion API. The models are saved inside the current directory. For more information on how to convert models, see this [page](https://docs.openvino.ai/2023.0/openvino_docs_model_processing_introduction.html).

In [ ]:
ov_model_xml_path = Path('models/ov_lraspp_model.xml')


if not ov_model_xml_path.exists():
    ov_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)
    # call tracing before converting. It is necessary in current version for segmentation models 
    traced_model = torch.jit.trace(model, dummy_input, strict=False)
    ov_model = ov.convert_model(traced_model, example_input=dummy_input)
    ov.save_model(ov_model, ov_model_xml_path)
else:
    print(f"IR model {ov_model_xml_path} already exists.")

## Show results
Confirm that the segmentation results look as expected by comparing model predictions on the OpenVINO IR and PyTorch models.

### Load an input image

Images need to be normalized before propagating through the network.

In [ ]:
import urllib.request

from torchvision.io import read_image
import torchvision.transforms as transforms

img_path = 'cats_image.jpeg'
urllib.request.urlretrieve(
    url='https://huggingface.co/datasets/huggingface/cats-image/resolve/main/cats_image.jpeg',
    filename=img_path
)
image = read_image(img_path)
display(transforms.ToPILImage()(image))

### Define a preprocessing and prepare an input data
You can use `torchvision.transforms` to make a preprocessing or use[preprocessing transforms from the model wight](https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models).

In [ ]:
import numpy as np


preprocess = models.segmentation.LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1.transforms()
preprocess.resize_size = (IMAGE_HEIGHT, IMAGE_WIDTH)  # change to image size

input_data = preprocess(image)
input_data = np.expand_dims(input_data, axis=0)

Select device from dropdown list for running inference using OpenVINO

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

### Run an inference on the OpenVINO model

In [ ]:
compiled_model = core.compile_model(ov_model_xml_path, device_name=device.value)

In [ ]:
res_ir = compiled_model(input_data)[0]

You can use [pytorch tutorial](https://pytorch.org/vision/0.12/auto_examples/plot_visualization_utils.html#sphx-glr-auto-examples-plot-visualization-utils-py) to visualize segmentation masks. Below is a simple example how to visualize the image with a `cat` mask.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F


plt.rcParams["savefig.bbox"] = 'tight'


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

Prepare and display a cat mask.

In [ ]:
sem_classes = [
    '__background__', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor'
]
sem_class_to_idx = {cls: idx for (idx, cls) in enumerate(sem_classes)}

normalized_mask = torch.nn.functional.softmax(torch.from_numpy(res_ir), dim=1)

cat_mask = normalized_mask[0, sem_class_to_idx['cat']]

show(cat_mask)

The [draw_segmentation_masks()](https://pytorch.org/vision/0.12/generated/torchvision.utils.draw_segmentation_masks.html#torchvision.utils.draw_segmentation_masks)function can be used to plots those masks on top of the original image. This function expects the masks to be boolean masks, but our masks above contain probabilities in [0, 1]. To get boolean masks, we can do the following:

In [ ]:
class_dim = 1
boolean_cat_mask = (normalized_mask.argmax(class_dim) == sem_class_to_idx['cat'])

And now we can plot a boolean mask on top of the original image.

In [ ]:
from torchvision.utils import draw_segmentation_masks

show(draw_segmentation_masks(image, masks=boolean_cat_mask, alpha=0.7, colors='yellow'))

### Run an inference on the PyTorch model to compare results

In [ ]:
model.eval()
with torch.no_grad():
    result_torch = model(torch.as_tensor(input_data).float())['out']


normalized_mask = torch.nn.functional.softmax(result_torch, dim=1)
boolean_cat_mask = (normalized_mask.argmax(class_dim) == sem_class_to_idx['cat'])
show(draw_segmentation_masks(image, masks=boolean_cat_mask, alpha=0.7, colors='yellow'))